### Imports

In [1]:
import pandas as pd
import cbpro
import datetime
from datetime import datetime as dt
import numpy as np
import time
import pickle
import configparser

In [2]:
## Bring in cbpro public client
public_client = cbpro.PublicClient()
products = public_client.get_products()

In [3]:
product_names = []
for i in products:
    product_names.append(i['id'])

In [4]:
config = configparser.ConfigParser()
config.read(r'C:\Users\hlu\repos\coinbase_v2\config_global.ini')

['C:\\Users\\hlu\\repos\\coinbase_v2\\config_global.ini']

### Parameters 

In [5]:
now = datetime.datetime.now()
end = now - datetime.timedelta(days=1)
start = end - datetime.timedelta(days=100)
candles = 200 # Max number of candles that can be ingested through API
instruments = product_names
granularity = config['PARAMETERS']['granularity']
if granularity =="minutes":
    granularity = 60

### Functions

In [6]:
#Function for pulling in data fromcbpro
def pull_data(gran_,start,end,candles,instrument):    #granularity [60, 300, 900, 3600, 21600, 86400]
    accepted_gran = [60, 300, 900, 3600, 21600, 86400]
    if gran_ not in accepted_gran:
        print("granularity wrong")
        return
    gran_mins = gran_/60
    gran_hours = gran_mins/60
    gran_days = gran_hours / 24
    duration = ((end-start)/gran_).seconds + ((end-start)/gran_).days*24*60*60
    print(duration)
    iterations = int(np.ceil(duration/200))
    new_df = pd.DataFrame()
    print("starting")
    print(iterations)
    for i in range(0,iterations):
        #print(i)
        time.sleep(0.2)
        if i ==0:
            end = end
            start = end-datetime.timedelta(minutes = gran_mins*200)
            v = public_client.get_product_historic_rates(instrument, start, end,  granularity=gran_)
            BTC_prices = pd.DataFrame(v)
            new_df = new_df.append(BTC_prices)
        else:
            end = end-datetime.timedelta(minutes = gran_mins*201)
            start = end-datetime.timedelta(minutes = gran_mins*200)
            v = public_client.get_product_historic_rates(instrument, start, end,  granularity=gran_)
            BTC_prices = pd.DataFrame(v)
            new_df = new_df.append(BTC_prices)
    print(new_df)
    new_df['time'] = new_df[0].apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    new_df = new_df.iloc[::-1].reset_index(drop=True)
    new_df = new_df.drop(columns =[0])
    new_df = new_df.rename(columns = {
                                            1:"low"
                                            ,2:"high"
                                            ,3:"open"
                                            ,4:"close"
                                            ,5:"volume"
                                            })
    return new_df

### Pull data

In [9]:
refined_ = []
for i in instruments:
    if ("BTC" in i)|("ETH" in i): 
        refined_.append(i)



In [17]:
"ETH-USD" in refined_

True

In [18]:
"BTC-USD" in refined_

True

In [19]:
"ETH-BTC" in refined_

True

In [20]:
for item in ["BTC-USD","ETH-USD","ETH-BTC"] :
    print(item)
    data = pull_data(granularity,start,end,candles,item)
    string_ = "minute_last_100_days_"+str(item)
    data.to_csv(string_+".csv")


BTC-USD
144000
starting
720
              0         1         2         3         4          5
0    1613416440  48634.50  48659.67  48648.00  48640.37   4.325478
1    1613416380  48633.59  48648.00  48639.67  48648.00   1.926474
2    1613416320  48619.44  48653.59  48653.59  48637.68  10.612604
3    1613416260  48636.54  48680.13  48671.59  48653.59   8.905345
4    1613416200  48636.32  48671.59  48641.94  48671.59   5.392603
..          ...       ...       ...       ...       ...        ...
195  1604733600  15495.00  15506.75  15496.43  15501.50   0.594267
196  1604733540  15496.43  15512.00  15511.99  15496.43   7.116215
197  1604733480  15508.13  15513.30  15513.23  15511.99   2.090170
198  1604733420  15510.39  15520.07  15520.06  15513.22   4.489967
199  1604733360  15520.06  15527.85  15527.31  15520.06   1.454774

[143980 rows x 6 columns]
ETH-USD
144000
starting
720
              0        1        2        3        4           5
0    1613416440  1823.64  1824.50  1824.50  1823.